In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown

In [ ]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/learnlm-1.5-pro-experimental
models/gemini-exp-1114
models/gemini-exp-1121


In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
import json

def retrieve_and_save_cfg(prompt, model, file_name="generated_cfg.json"):
    """
    Retrieve CFG production rules from a Generative AI tool and save them in JSON format.

    Args:
        prompt (str): The prompt sent to the Generative AI tool.
        model: The generative model object (e.g., from google.generativeai).
        file_name (str): The name of the JSON file to save the data.

    Returns:
        dict: The AI-generated CFG production rules.
    """
    try:
        # Generate content using the AI model
        response = model.generate_content(prompt)
        response_text = response.text.strip()

        # Prepare the data for saving
        data = {"prompt": prompt, "CFG_rules": response_text}

        # Save to a JSON file
        with open(file_name, 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"\nResponse successfully saved to {file_name}")

        print("\nGenerated CFG Production Rules (Readable Format):\n")
        print(f"Prompt: {prompt}")
        print("\nCFG Rules:\n")
        print(response_text)

        return data
    except Exception as e:
        print(f"Error retrieving or saving data: {e}")
        return None

In [ ]:
def load_hardcoded_cfg(option):
    hardcoded_cfg = {
        "Arithmetic Expressions": {
            "S": ["T", "T + S", "T - S"],
            "T": ["F", "F * T", "F / T"],
            "F": ["( S )", "number"]
        },
        "Control Structures": {
            "stmt": ["if expr then stmt else stmt", "while expr do stmt", "assign"],
            "assign": ["identifier = expr"],
            "expr": ["expr && term", "expr || term", "term"],
            "term": ["!factor", "factor"],
            "factor": ["true", "false", "( expr )", "identifier"]
        },
        "Function Definitions and Calls": {
            "function": ["func identifier ( params ) { stmts }"],
            "params": ["identifier", "identifier , params", "ε"],
            "stmts": ["stmt ; stmts", "ε"],
            "call": ["identifier ( args )"],
            "args": ["expr", "expr , args", "ε"]
        },
        "Assignment Operations": {
            "assign": ["identifier = expr"],
            "expr": ["identifier + identifier", "identifier - identifier", "number"],
            "number": ["digit number", "digit"],
            "digit": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
        },
        "Boolean Expressions": {
            "bool_expr": ["bool_expr && bool_term", "bool_term"],
            "bool_term": ["bool_term || bool_factor", "bool_factor"],
            "bool_factor": ["!bool_atom", "bool_atom"],
            "bool_atom": ["true", "false", "( bool_expr )", "identifier"]
        },
        "Loop Structures": {
            "loop": ["for ( assign ; expr ; assign ) stmt"],
            "stmt": ["assign ; stmt", "if expr then stmt else stmt", "while expr do stmt"],
            "expr": ["identifier < identifier", "identifier > identifier"]
        },
        "Array Operations": {
            "array_decl": ["type identifier [ number ]"],
            "array_assign": ["identifier [ expr ] = expr"],
            "expr": ["expr + term", "term"],
            "term": ["factor * term", "factor"],
            "factor": ["number", "identifier"]
        },
        "String Operations": {
            "str_expr": ["str_expr + str_term", "str_term"],
            "str_term": ['"string_literal"', "identifier"],
            "assign": ["identifier = str_expr"]
        },
        "Basic Mathematical Functions": {
            "expr": ["sin ( term )", "cos ( term )", "log ( term )", "term"],
            "term": ["term * factor", "term / factor", "factor"],
            "factor": ["number", "identifier", "( expr )"]
        },
        "Conditional Expressions with Ternary Operator": {
            "ternary_expr": ["expr ? expr : expr"],
            "expr": ["identifier < identifier", "identifier > identifier", "expr + term", "term"],
            "term": ["factor * term", "factor"],
            "factor": ["number", "identifier"]
        }
    }
    return hardcoded_cfg.get(option, {})

In [ ]:
prompt = input("Input: ")
cfg_data = retrieve_and_save_cfg(prompt, model, "arithmetic_expressions.json")
print("Retrieved CFG Production Rules:")
print(cfg_data)

Input: Generate the CFG production rules for Arithmetic Expressions

Response successfully saved to arithmetic_expressions.json

Generated CFG Production Rules (Readable Format):

Prompt: Generate the CFG production rules for Arithmetic Expressions

CFG Rules:

**Production Rules for Arithmetic Expressions (E):**

1. E -> T
2. E -> E + T
3. E -> E - T
4. E -> E * T
5. E -> E / T
6. T -> n (where n is a numeric literal)
7. T -> ( E )
Retrieved CFG Production Rules:
{'prompt': 'Generate the CFG production rules for Arithmetic Expressions', 'CFG_rules': '**Production Rules for Arithmetic Expressions (E):**\n\n1. E -> T\n2. E -> E + T\n3. E -> E - T\n4. E -> E * T\n5. E -> E / T\n6. T -> n (where n is a numeric literal)\n7. T -> ( E )'}


In [ ]:
from google.colab import files

files.download('arithmetic_expressions.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>